In [21]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import os

In [22]:
import glob
import os

# Define the path to the directory containing the files
directory_path = "./nested_squares/"

# Create a pattern to match files recursively
pattern = os.path.join(directory_path, "**/*claude*.md")

# Use glob to find all files that match the pattern, with recursive set to True
md_files = glob.glob(pattern, recursive=True)

# Read the content of each file
file_contents = {}
for file_path in md_files:
    with open(file_path, "r") as file:
        file_contents[os.path.basename(file_path)] = file.read()

In [23]:
gt_data = pd.read_json(f"./nested_squares/config.json")
len(gt_data)

120

In [24]:
# text_image_0a5a0d82-477d-47e2-a504-861c5952fe98-gemini-output.md
gt_data["claude-output-file"] = gt_data["image_name"].apply(
    lambda x: "./nested_squares/"
    + x.replace(".png", "")
    + "-claude-3-sonnet-20240229-output.md"
)

# Read the content of each file for the gemini output and make a new column, ignoring missing files
gt_data["claude-output-raw"] = gt_data["claude-output-file"].apply(
    lambda x: (open(x, "r").read() if os.path.exists(os.path.join(x)) else None)
)

In [25]:
# drop rows with missing gemini output
gt_data = gt_data.dropna(subset=["claude-output-raw"])

In [26]:
len(gt_data)

120

In [27]:
import re


def extract_marked_text(text):
    # Map textual numbers to integer values
    number_mapping = {
        "zero": 0,
        "one": 1,
        "two": 2,
        "three": 3,
        "four": 4,
        "five": 5,
        "six": 6,
        "seven": 7,
        "eight": 8,
        "nine": 9,
        "ten": 10,
    }

    # Use regular expression to find all numbers in the text
    numbers = re.findall(r"\b\d+\b", text)
    if numbers:
        # Convert the first found number to integer
        return int(numbers[0])

    # If no digits found, try to find textual numbers
    words = text.split()
    for word in words:
        if word.lower() in number_mapping:
            return number_mapping[word.lower()]

    # If no number is found, return a default value indicating no number was found
    return "number_not_found"


gt_data["predicted"] = gt_data["claude-output-raw"].apply(extract_marked_text)

In [28]:
gt_data["claude-output-raw"]

0      The image consists of two squares, one inside ...
1      The image contains 2 squares - an outer larger...
2      In the image, there are 2 squares visible. The...
3      The image contains 2 squares nested inside eac...
4      The image contains 2 squares. There is a large...
                             ...                        
115    In the given image, there are 5 squares visibl...
116    The image contains a series of concentric squa...
117    The image consists of 5 concentric squares dra...
118    The image depicts a series of concentric squar...
119    The image contains a series of nested square s...
Name: claude-output-raw, Length: 120, dtype: object

In [29]:
gt_data["predicted"].value_counts()

predicted
4    58
2    27
5    27
3     8
Name: count, dtype: int64

In [30]:
cleaned_data = gt_data.copy()
# drop squares column
cleaned_data = cleaned_data.drop(columns=["squares"])

In [31]:
# Ensure both columns are of integer type before comparison
cleaned_data["depth"] = cleaned_data["depth"].astype(int)
cleaned_data["predicted"] = cleaned_data["predicted"].astype(int)

cleaned_data["is_prediction_correct"] = (
    cleaned_data["depth"] == cleaned_data["predicted"]
)
# Calculate accuracy
accuracy = cleaned_data["is_prediction_correct"].mean()
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 69.17%


In [32]:
len(cleaned_data)

120

In [33]:
# Assuming 'cleaned_data' DataFrame has columns 'line_thickness', 'num_intersections', and 'extracted_number'

# Convert line_thickness to an appropriate numeric type if necessary
cleaned_data["line_thickness"] = cleaned_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = cleaned_data.groupby("line_thickness").apply(
    lambda df: (df["is_prediction_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_13119/1445926420.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = cleaned_data.groupby("line_thickness").apply(


line_thickness
2    62.0
3    65.0
4    80.0
dtype: float64

In [34]:
# Convert line_thickness to an appropriate numeric type if necessary
cleaned_data["depth"] = cleaned_data["depth"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = cleaned_data.groupby("depth").apply(
    lambda df: (df["is_prediction_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_13119/3496425564.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = cleaned_data.groupby("depth").apply(


depth
2    90.0
3    27.0
4    87.0
5    73.0
dtype: float64